<a href="https://colab.research.google.com/github/marcoplacenti/protein_generation/blob/main/mini_progen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Mini-progen

## A mini-progen implementation using the transformers library and the GPT-2 architecture.

## 1. Install libraries



In [3]:
!pip install transformers
!pip install tokenizers
# versions when implemented: tokenizers-0.9.3 transformers-3.5.1

     |████████████████████████████████| 1.3MB 9.1MB/s 
     |████████████████████████████████| 1.1MB 30.8MB/s 
     |████████████████████████████████| 2.9MB 48.2MB/s 
     |████████████████████████████████| 890kB 53.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=a1518a4ab2d773beaeddf6136cc305f8a7faadec6f9663b3c42a2584e56930cb
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


##2. Download dataset from pfam

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
ls

drive/  sample_data/


In [6]:
file_name = "/content/drive/MyDrive/Colab Notebooks/project/peptides.txt"

In [7]:
from tokenizers import ByteLevelBPETokenizer

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train (  files=file_name, vocab_size=2050, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>"],
    show_progress=True)

In [8]:
tokenizer.save("vocab.json")

In [9]:
ls

drive/  sample_data/  vocab.json


In [10]:
from transformers import GPT2Model, GPT2Config,GPT2LMHeadModel
configuration = GPT2Config()
model = GPT2LMHeadModel(configuration)
print(configuration)
print(model)

GPT2Config {
  "activation_function": "gelu_new",
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "resid_pdrop": 0.1,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "vocab_size": 50257
}

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout):

In [14]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast(tokenizer_file="vocab.json",
                                    model_max_length=40,
                                    pad_token="<pad>",
                                    mask_token="<mask>",
                                    sep_token="<sep>",
                                    padding=True,
                                    truncation=True)


In [15]:
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(  
    tokenizer=tokenizer,
    file_path=file_name,
    block_size=100,
)

/usr/local/lib/python3.6/dist-packages/transformers/data/datasets/language_modeling.py:114: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


In [16]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [17]:
import torch
torch.cuda.is_available()

True

In [21]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="\mini-progen",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_gpu_train_batch_size=64,
    save_steps=10000,
    save_total_limit=2,
)

trainer = Trainer( 
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator,
    prediction_loss_only=True,  
)

/usr/local/lib/python3.6/dist-packages/transformers/trainer.py:281: FutureWarning: Passing `prediction_loss_only` as a keyword argument is deprecated and won't be possible in a future version. Use `args.prediction_loss_only` instead. Setting `args.prediction_loss_only=True
  FutureWarning,


In [22]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Step,Training Loss


CPU times: user 1min 34s, sys: 1min 2s, total: 2min 37s
Wall time: 2min 38s


TrainOutput(global_step=310, training_loss=1.2268387333039315)

In [20]:
from transformers import pipeline
text_generator = pipeline("text-generation",
                          model = model,
                          tokenizer = tokenizer,
                          )
text_generator("H")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


RuntimeError: ignored